*****************************************************
 # The Social Web 				    
  2016-2017 Master Information Sciences	    
  Instructors Davide Ceolin, Cristina Bucur 	            
  Exercises for Hands-on session 3		    
  23 February 2017 11:00 - 12:45                   
  HG-0B-16, HG-0B-08  
*****************************************************

Prerequisites:
- Python 2.6 or 2.7
- Python packages: requests, BeautifulSoup, HTMLParser, rdflib, rdflib_microdata*
-------------------------------------------------------------------------------
* NB! You have to install the rdflib_microdata package from Git, as it is not in the standard PIP library. You can use: 
$ pip install -e git+https://github.com/edsu/rdflib-microdata.git#egg=rdflib-microdata 

IF THAT DOES NOT WORK: (Command git not found): 

- Download the package manually as zip file from https://github.com/edsu/rdflib-microdata#egg=rdflib-microdata 
- unpack the zip file in the folder you created for The Social Web (H:\socialweb in the getting started guide). In your social web folder, do:

* pip install six

* pip install rdflib-microdata-master\ 

* (if there is an error with 'setuptools', uninstall and re-instal them (pip uninstall setuptools, pip install setuptools) and try again)

-------------------------------------------------------------------------------

In this session you are going to mine data in various microformats. You will see the differences in what each of the formats can contain and what purpose they serve. We will start by looking at geographical data. 

Exercise 1: Extracting coordinates from a webpage and reformatting them in the geo microformat (based on Example 8-1 in Mining the Social Web). Save and run the following code as a Python script.

In [ ]:
# -*- coding: utf-8 -*-
!pip install -e git+https://github.com/edsu/rdflib-microdata.git#egg=rdflib-microdata

import sys
import requests
from BeautifulSoup import BeautifulSoup

# This script requires you to add a url of a page with geotags to the commandline, e.g.
# python geo.py "http://en.wikipedia.org/wiki/Amsterdam"
URL = sys.argv[1]

req = requests.get(URL, headers={'User-Agent' : "Social Web Course Student"})
soup = BeautifulSoup(req.text)

geoTag = soup.find(True, 'geo')

if geoTag and len(geoTag) > 1:
        lat = geoTag.find(True, 'latitude').string
        lon = geoTag.find(True, 'longitude').string
        print 'Location is at', lat, lon
elif geoTag and len(geoTag) == 1:
        (lat, lon) = geoTag.string.split(';')
        (lat, lon) = (lat.strip(), lon.strip())
        print 'Location is at', lat, lon
else:
        print 'No location found'

-------------------------------------------------------------------------------
Task 1: Can you convert the output of Exercise 1 into KML? Here is the KML documentation: https://developers.google.com/kml/documentation/?csw=1 and here you can find a simple example of how it is used: https://developers.google.com/kml/documentation/KML_Samples.kml

Visualise the point in Google Maps using the following code example: https://developers.google.com/maps/documentation/javascript/examples/layer-kml-features
You will have to create your own KML file for the custom map layer, and provide a URL to the KML file inside the JavaScript code, which means that you have to upload the file somewhere. You can use a service like http://pastebin.com/ to obtain a URL for your KML file —> paste the code there and request the RAW format URL; use this one in this Task1.

Is KML a microformat, why (not)?

-------------------------------------------------------------------------------

There are a lot of specialised microformats, such as hRecipe to format recipes in. This format helps search engines to find recipes and index them properly so when you search for "recipe biscotti" it knows that it needs to return a page that contains ingredients and instructions. 

Exercise 2: Parsing hRecipe data for a Chocolate Biscotti recipe (from Example 2-7 in Mining the Social Web):

In [ ]:
# -*- coding: utf-8 -*-

import sys
import requests
import json
import BeautifulSoup

# Pass in a URL containing hRecipe, such as
# http://wholewheatsweets.com/recipe/cookies/whole_wheat_chocolate_hazelnut_biscotti

URL = sys.argv[1]

# Parse out some of the pertinent information for a recipe.
# See http://microformats.org/wiki/hrecipe.

def parse_hrecipe(url):
	req = requests.get(URL)
	
	soup = BeautifulSoup.BeautifulSoup(req.text)
	
	hrecipe = soup.find(True, 'hrecipe')
	
	if hrecipe and len(hrecipe) > 1:
		fn = hrecipe.find(True, 'fn').string
		yield_ = hrecipe.find(True, 'yield').find(text=True)
		ingredients = [i.string
			for i in hrecipe.findAll(True, 'ingredient')
				if i.string is not None]
	
		instructions = []
		for i in hrecipe.find(True, 'instructions').findAll(True, 'instruction'):
			if type(i) == BeautifulSoup.Tag:
				s = ''.join(i.findAll(text=True)).strip()
			elif type(i) == BeautifulSoup.NavigableString:
				s = i.string.strip()
			else:
				continue
	
			if s!='':
				instructions += [s]
	
		return {
			'name': fn,
			'yield:': yield_,
			'ingredients': ingredients,
			'instructions': instructions,
			}
	else:
		return {}

recipe = parse_hrecipe(URL)
print json.dumps(recipe, indent=4)

-------------------------------------------------------------------------------

Task 2: Can you modify the hRecipe script in such a way that it gives a more informative error message if no recipe information is found instead of {}

-------------------------------------------------------------------------------

Task 3: Does the hRecipe format facilitate easy comparison of different recipes? Can you for example easily compare different dessert recipe ingredients? For inspiration you can look back at the exercises you did in Hands-on session 1 where you compared different sets of tweets.

-------------------------------------------------------------------------------

As you might have noticed in the previous exercise, hRecipe is not used on many sites anymore, instead Schema.org annotations are added. Schema.org is a multipurpose format that has been created by a consortium consisting of Yahoo!, Google and Microsoft. It can describe entities, events, products etc. Check out the vocabulary specs on Schema.org.

Exercise 3: Parsing schema.org microdata. To parse this data you need to install the rdflib-microdata package, which is not in the standard pip repository. You can do so as follows:

In [ ]:
!pip install -e git+https://github.com/edsu/rdflib-microdata.git#egg=rdflib-microdata

Find some example pages containing microdata [here](https://github.com/LawrenceWoodman/mida/wiki/Sites-Using-Microdata).

In [ ]:
# -*- coding: utf-8 -*-

import sys
import rdflib
import rdflib_microdata

# Pass in a URL containing Schema.org microformats
# https://ephesusbreeze.com/tours/kusadasi-ephesus
url = sys.argv[1]

g = rdflib.Graph()
g.parse(url, format="microdata")
print g.serialize()

-------------------------------------------------------------------------------

Task 4: Compare the schema.org information about a band on last.fm to the Facebook Open Graph information about the same band from Facebook. What are the differences? Which of the formats supports better interoperability do you think?

-------------------------------------------------------------------------------

-------------------------------------------------------------------------------

Task 5: Explore the various microformats that http://microform.at can handle and compare the output of the exercises with the output of microform.at Think about possible microformats you want to support in your final assignment and read up on how to parse them.

-------------------------------------------------------------------------------